# AI Community @ Семинар № 4, весна 2018
## Машинный перевод: Практика

In [11]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import time

use_cuda = torch.cuda.is_available()

In [25]:
from util import prepareData, variablesFromPair, timeSince, evaluate
from util import SOS_token, EOS_token, MAX_LENGTH

Загрузим данные

In [13]:
input_lang, output_lang, pairs = prepareData('eng', 'deu', True)

Reading lines...
Read 115245 sentence pairs
Trimmed to 7400 sentence pairs
Counting words...
Counted words:
spa 3908
eng 2788


## Кодировщик

![](./images/encoder.png)

In [14]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

## Декодировщик

![](./images/decoder.png)

In [15]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

## Внимательный декодировщик

![](./images/attention.png)

![](./images/attention_decoder.png)

In [16]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

## Обучение

In [17]:
teacher_forcing_ratio = 0.5

def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_variable[di])
            decoder_input = target_variable[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]

            decoder_input = Variable(torch.LongTensor([[ni]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input

            loss += criterion(decoder_output, target_variable[di])
            if ni == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0] / target_length

In [18]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [variablesFromPair(random.choice(pairs), input_lang, output_lang)
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_variable = training_pair[0]
        target_variable = training_pair[1]

        loss = train(input_variable, target_variable, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [23]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1)


if use_cuda:
    encoder1 = encoder1.cuda()
    attn_decoder1 = attn_decoder1.cuda()

trainIters(encoder1, attn_decoder1, 75000, print_every=100)

0m 12s (- 154m 42s) (100 0%) 4.8299
0m 22s (- 141m 42s) (200 0%) 3.8611
0m 32s (- 136m 8s) (300 0%) 3.6888
0m 42s (- 131m 19s) (400 0%) 3.5348
0m 51s (- 127m 49s) (500 0%) 3.4841
1m 2s (- 128m 36s) (600 0%) 3.5849
1m 12s (- 127m 26s) (700 0%) 3.5351
1m 21s (- 126m 18s) (800 1%) 3.5864
1m 31s (- 125m 17s) (900 1%) 3.3020
1m 40s (- 124m 17s) (1000 1%) 3.4305
1m 50s (- 124m 1s) (1100 1%) 3.5243
2m 1s (- 124m 11s) (1200 1%) 3.4385
2m 10s (- 123m 40s) (1300 1%) 3.4343
2m 21s (- 124m 7s) (1400 1%) 3.2032
2m 32s (- 124m 40s) (1500 2%) 3.3740
2m 42s (- 124m 6s) (1600 2%) 3.0851
2m 52s (- 124m 3s) (1700 2%) 3.1853
3m 2s (- 123m 38s) (1800 2%) 3.2106
3m 12s (- 123m 33s) (1900 2%) 3.1471
3m 23s (- 123m 46s) (2000 2%) 3.2703
3m 33s (- 123m 44s) (2100 2%) 3.1366
3m 44s (- 123m 45s) (2200 2%) 3.1070
3m 54s (- 123m 27s) (2300 3%) 3.0135
4m 4s (- 123m 14s) (2400 3%) 3.0649
4m 14s (- 123m 7s) (2500 3%) 3.0224
4m 24s (- 122m 58s) (2600 3%) 2.9793
4m 35s (- 122m 47s) (2700 3%) 3.0199
4m 45s (- 122m 34s) 

KeyboardInterrupt: 

In [31]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [38]:
w, _ = evaluate(encoder1, attn_decoder1, 'habla', input_lang, output_lang)
w

['i', 'm', 'sorry', 'to', 'the', 'him', '.', '<EOS>']

## Сохранение и загрузка модели

In [39]:
torch.save(encoder1.state_dict(), "./encoder")
torch.save(attn_decoder1.state_dict(), "./decoder")

In [40]:
encoder2 = EncoderRNN(input_lang.n_words, hidden_size)
attn_decoder2 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1)
encoder2.load_state_dict(torch.load("./encoder"))
attn_decoder2.load_state_dict(torch.load("./decoder"))

In [41]:
evaluateRandomly(encoder2, attn_decoder2)

['es el consejero de mayor confianza del rey .', 'he s the king s most trusted advisor .']


# TODO рисование внимания, loss

Источники:
1. [Pytorch seq2seq tutorial](http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)